<div style="position: relative; text-align: center; padding: 30px;">
  <h1><strong>Ejercicios de Modelado</strong></h1>
  <h3><strong>Ejercicio 4</strong></h3>
</div>

Supongamos una situación donde se tienen $n$ trabajos y $m$ máquinas y cada trabajo debe procesarse en cada máquina. Para cada trabajo, el orden de procesamiento en las máquinas es fijo, es decir, el trabajo $j$ debe procesarse primero en la máquina $j(1)$ y luego en la máquina $j(2)$, y así sucesivamente. Una máquina solo puede procesar un trabajo a la vez, y una vez que se inicia un trabajo en cualquier máquina, debe procesarse hasta su finalización. 

El objetivo es minimizar la suma de los tiempos de finalización de todos los trabajos. Los datos que especifican una instancia del problema son $m$, $n$ y $p_{ij}$ para $j = 1, \dots, n$ y $i = 1, \dots, m$, que es el tiempo de procesamiento del trabajo $j$ en la máquina $i$, y el orden de procesamiento en las máquinas, $j(1), \dots, j(m)$, para el trabajo $j$, $j = 1, \dots, n$. Formule el problema.  

Sean $t_{ij}$, $i = 1, \dots, m$, $j = 1, \dots, n$, variables de decisión que denotan el tiempo de inicio del trabajo $j$ en la máquina $i$ y sean también $y_{ijk}$, $i = 1, \dots, m$, $j, k = 1, \dots, n : j < k$, variables binarias que toman el valor de 1 si el trabajo $j$ precede al trabajo $k$ en la máquina $i$ y 0 en otro caso.  


In [6]:
from ortools.linear_solver import pywraplp

In [7]:
solver = pywraplp.Solver.CreateSolver('SCIP')

### **Conjunto de índices**

- $N$: Conjunto de trabajos, $C = \{1, 2, \dots, 7\}$
- $M$: Conjunto de máquinas, $P = \{1, 2, \dots, 3\}$

In [8]:
m = 4 # número de máquinas
n = 8 # número de trabajos
N = range(n)  # trabajos 
M = range(m)  # máquinas

### **Parámetros**

- $ \forall i \in M, \forall j \in N: p_{i,j} = \text {tiempos de procesamiento} $
- $ \forall j \in N: orders_{j} = \text {orden de procesamiento para el trabajo j} $

In [ ]:
#matriz de tiempos de procesamiento p[i][j] para i = 0,...,3 y j = 0,...,7
p = [
    [10, 5, 7, 9, 6, 12, 11, 4],
    [6, 6, 4, 10, 3, 12, 11, 5],
    [9, 7, 6, 8, 4, 2, 3, 3],
    [4, 6, 8, 3, 6, 9, 2, 1]
]

#orden de procesamiento de cada trabajo
#cada lista indica, para el trabajo j, el orden de las máquinas.
orders = [
    [2, 1, 3, 0],  # Trabajo 1: 3 2 4 1
    [0, 1, 3, 2],  # Trabajo 2: 1 2 4 3
    [0, 1, 2, 3],  # Trabajo 3: 1 2 3 4
    [3, 0, 2, 1],  # Trabajo 4: 4 1 3 2
    [3, 0, 1, 2],  # Trabajo 5: 4 1 2 3
    [2, 0, 1, 3],  # Trabajo 6: 3 1 2 4
    [2, 0, 3, 1],  # Trabajo 7: 3 1 4 2
    [2, 1, 0, 3],  # Trabajo 8: 3 2 1 4
]


#cota superior p
MM = 1000 # es MM porque M es el número de máquinas :P

### **Variables de decisión**

In [10]:
# Crear variables:
# t[i, j]: tiempo de inicio del trabajo j en la máquina i.
t = {}
for i in M:
    for j in N:
        t[(i, j)] = solver.NumVar(0, solver.infinity(), f't_{i}_{j}') # t es un número real no negativo

# y[i, j, k]: variable binaria que es 1 si en la máquina i el trabajo j precede al trabajo k (para j < k)
y = {}
for i in M:
    for j in N:
        for k in range(j + 1, n):
            y[(i, j, k)] = solver.BoolVar(f'y_{i}_{j}_{k}')


### **Función objetivo**

Minimizar la suma de los tiempos de inicio en la última máquina para cada trabajo

$$
\text{Minimizar} \sum_{j=1}^{m} t_{j(m),j}
$$

In [11]:
tiempo = solver.Objective()

tiempo = sum([t[(orders[j][-1], j)] for j in N])

solver.Minimize(tiempo)

La última máquina para el trabajo $j$ es $orders[j][-1]$

### **Restricciones**

$$
t_{ij} \geq 0, \quad i = 1, . . . , m, \quad j = 1, . . . , n
$$

$$
y_{ijk} \in \{0, 1\}, \quad i = 1, . . . , m, \quad j, k = 1, . . . , n : j < k
$$

Orden de procesamiento de cada trabajo

$$
t_{j(r+1),j} \geq t_{j(r),j} + p_{j(r),j}, \quad r = 1, . . . , m − 1, \quad j = 1, . . . , n
$$

In [12]:
for j in N:
    mach_order = orders[j] # secuencia de máquinas para el trabajo j
    for r in range(m - 1): # para cada máquina en la secuencia
        i_r = mach_order[r]     # máquina en la posición r para el trabajo j
        i_next = mach_order[r + 1]  # siguiente máquina en la secuencia
        solver.Add(t[(i_next, j)] >= t[(i_r, j)] + p[i_r][j])


Restricciones de solapamiento en cada máquina

$$
t_{ij} + p_{ij} \leq t_{ik} + M (1 − y_{ijk}), \quad i = 1, . . . , m, \quad j, k = 1, . . . , n : j < k
$$

In [13]:
for i in M:
    for j in N:
        for k in range(j + 1, n):
            #si el trabajo j se procesa antes que el trabajo k en la máquina i:
            solver.Add(t[(i, j)] + p[i][j] <= t[(i, k)] + MM * (1 - y[(i, j, k)]))

$$
t_{ik} + p_{ik} \leq t_{ij} + M y_{ijk}, \quad i = 1, . . . , m, \quad j, k = 1, . . . , n : j < k
$$

In [14]:
for i in M:
    for j in N:
        for k in range(j + 1, n): # si el trabajo k se procesa antes que el trabajo j en la misma máquina
            #si el trabajo k se procesa antes que el trabajo j en la misma máquina:
            solver.Add(t[(i, k)] + p[i][k] <= t[(i, j)] + MM * y[(i, j, k)])

### **Resolver**

In [15]:
status = solver.Solve()

In [16]:
print('Tiempo:', solver.Objective().Value())

Tiempo: 292.0000000000002


In [ ]:
print("\nTiempos de inicio por máquina y trabajo:")
for i in M:
    for j in N:
        print(f"t[{i}][{j}] = {t[(i, j)].solution_value()}", end="; ")
    print() # nueva línea


Tiempos de inicio por máquina y trabajo:
t[0][0] = 42.00000000000007; t[0][1] = 16.999999999999964; t[0][2] = 0.0; t[0][3] = 33.00000000000007; t[0][4] = 6.999999999999943; t[0][5] = 51.99999999999994; t[0][6] = 21.999999999999986; t[0][7] = 12.999999999999979; 
t[1][0] = 31.000000000000128; t[1][1] = 21.999999999999968; t[1][2] = 7.999999999999972; t[1][3] = 50.00000000000008; t[1][4] = 15.0; t[1][5] = 63.99999999999995; t[1][6] = 37.00000000000024; t[1][7] = 2.999999999999972; 
t[2][0] = 22.000000000000128; t[2][1] = 33.99999999999992; t[2][2] = 11.999999999999973; t[2][3] = 42.00000000000007; t[2][4] = 18.0; t[2][5] = 31.999999999999922; t[2][6] = 3.000000000000007; t[2][7] = -2.8094592911604636e-14; 
t[3][0] = 37.00000000000013; t[3][1] = 27.999999999999925; t[3][2] = 17.99999999999997; t[3][3] = 6.000000000000165; t[3][4] = -2.7305351989126187e-11; t[3][5] = 75.99999999999994; t[3][6] = 35.00000000000024; t[3][7] = 16.99999999999998; 
